In [1]:
import sys
sys.path.append('..')

import pandas as pd
from data_center.binance_handler_v2 import BinanceHandHandler

In [2]:
BinanceHandHandler().endpoint

KeyError: 'basic_url'

In [6]:
import requests
import time

url = "https://fapi.binance.com"
# Base URL for Binance API
BASE_URL = "https://fapi.binance.com"

# Endpoint for the funding rate
FUNDING_RATE_ENDPOINT = "/fapi/v1/fundingRate"

# Set the symbol for which you want to get the funding rate
symbol = "BTCUSDT"

# Optionally, set the start and end times for the funding rate history in milliseconds
# For example, getting data for the last 24 hours
end_time = int(time.time() * 1000)  # Current time in milliseconds
start_time = end_time - 24*60*60*1000  # 24 hours before the current time

# Making the GET request to the Binance API
response = requests.get(BASE_URL + FUNDING_RATE_ENDPOINT, params={
    "symbol": symbol,
    "startTime": start_time,
    "endTime": end_time
})

# Checking if the request was successful
if response.status_code == 200:
    # Parsing the response to JSON
    data = response.json()

    new_df = pd.DataFrame(data)

    new_df['fundingTime'] = pd.to_datetime(new_df['fundingTime'], unit='ms')
    new_df['datetime'] = new_df['datetime'].dt.strftime('%Y-%m-%d %H:%M:%S')
    new_df = new_df.drop('symbol',axis =1)

else:
    print("Failed to fetch data: Status code", response.status_code)


KeyError: 'datetime'

In [16]:
df = pd.DataFrame(data)

df['fundingTime'] = pd.to_datetime(df['fundingTime'], unit='ms')
df.drop
print(df)

    symbol         fundingTime fundingRate       markPrice
0  BTCUSDT 2024-04-19 08:00:00  0.00006641  64517.90000000
1  BTCUSDT 2024-04-19 16:00:00  0.00005880  64425.67251773
2  BTCUSDT 2024-04-20 00:00:00  0.00010000  63787.40000000


In [17]:
df.drop('symbol',axis =1)
new_df['datetime'] = new_df['datetime'].dt.strftime('%Y-%m-%d %H:%M:%S')

,fundingTime,fundingRate,markPrice
0,2024-04-19 08:00:00,0.00006641,64517.90000000
1,2024-04-19 16:00:00,0.00005880,64425.67251773
2,2024-04-20 00:00:00,0.00010000,63787.40000000
